In [2]:
import numpy as np
import os
import scipy.ndimage
from skimage.feature import hog
from skimage import data, color, exposure
from sklearn.model_selection import  train_test_split
from sklearn.neighbors import KNeighborsClassifier
import joblib
import cv2
import imageio.v2 as imageio

In [4]:

features_list = []
features_label = []
# load labeled training / test data
# loop over the 10 directories where each directory stores the images of a digit
for digit in range(0,10):
    label = digit
    training_directory = '../../Training/Dataset/digits/' + str(label) + '/'
    print(len(os.listdir(training_directory)))
    for filename in os.listdir(training_directory):
        image = cv2.imread(training_directory + filename, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (25, 25))

        # get the HOG descriptor for the image
        hog_desc = hog(image, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
        features_label.append(label)
        features_list.append(hog_desc)

23
15
30
27
29
33
32
16
19
30


In [6]:
# store features array into a numpy array
features  = np.array(features_list, 'float64')
# split the labled dataset into training / test sets
X_train, X_test, y_train, y_test = train_test_split(features, features_label,test_size=0.1)
# train using K-NN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
# get the model accuracy
model_score = knn.score(X_test, y_test)
print(model_score)
# save trained model
joblib.dump(knn, './models/knn_model.pkl')

0.9230769230769231


['./models/knn_model.pkl']

In [7]:
knn = joblib.load('./models/knn_model.pkl')

# extract the features from the image to predict
def feature_extraction(image):
    return hog(image, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')


def predict(df):
    predict = knn.predict(df.reshape(1,-1))[0]
    predict_proba = knn.predict_proba(df.reshape(1,-1))
    return predict, predict_proba[0][predict]
digits = []

# load your image from file
image = cv2.imread('../../Training/Dataset/digits/4/Cell_14_Test_11.jpg' , cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (25, 25))
digits.append(image)
# extract featuress
hogs = list(map(lambda x: feature_extraction(x), digits))
# apply k-NN model created in previous
predictions = list(map(lambda x: predict(x), hogs))


print(predictions[0])

(3, 1.0)
